# Aplicacion Codebar con Flask

In [1]:
# librerias

from flask import Flask, render_template, Response, request, Markup

import cv2
from pyzbar.pyzbar import decode
from PIL import Image
import pandas as pd
import ast
import pymongo
import requests
import matplotlib.pyplot as plt
%matplotlib inline


import warnings
warnings.simplefilter('ignore')

In [2]:
def captura():
    cam=cv2.VideoCapture(0)
    while 1:
        ret, frame=cam.read()
        if ret==False: break
            
        frame=cv2.imencode('.jpg', frame)[1].tobytes()
        yield (b'--frame\r\n'b'Content-Type: image/jpeg\r\n\r\n' + frame + b'\r\n')

In [3]:
def decodifica():
    codigo=decode(Image.open('capturas/codigo.png'))[0][0]
    return codigo

In [4]:
def elemento(df, codigo):
    
    elem=df.loc[df.ean==str(ast.literal_eval(codigo.decode('utf8')))[0:-1]]

    res=requests.get(elem.URL[elem.index[0]], stream=True)
    img=Image.open(res.raw)
    
    return elem, img

In [5]:
def busqueda_similares(df, elem, campo):
    
    productos=df.loc[df[campo]==elem[campo][elem.index[0]]]
    
    return productos

In [6]:
app=Flask(__name__)

In [7]:
df=None

@app.before_first_request  
def startup():
    global df
    df=pd.read_csv('data.csv')

In [8]:
@app.route('/video')
def video():
    return Response(captura(), mimetype='multipart/x-mixed-replace; boundary=frame')

In [9]:
@app.route('/', methods=['POST', 'GET'])
def main():
    return render_template('main.html')

In [10]:
@app.route('/proceso', methods=['POST', 'GET'])
def proceso():
    if request.method=='POST':

        cam=cv2.VideoCapture(0)
        ret, frame=cam.read()
        cam.release()
        
        img_name='capturas/codigo.png'
        cv2.imwrite(img_name, frame)
        
        codigo=decodifica()
        
        elem, img=elemento(df, codigo)
        
        try: 
            plot_url=base64.b64encode(img.getvalue()).decode()
        except:
            with open('static/images/error.png', 'rb') as f:
                plot_url=base64.b64encode(f.read()).decode()

        return render_template('plots.html',
                               model_plot=Markup('<img src="data:image/png;base64,{}">'.format(plot_url)))

    else:
        return render_template('plots.html',
                                model_plot='')

In [ ]:
if __name__=='__main__':
    app.run(debug=False)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [15/Aug/2020 22:50:05] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [15/Aug/2020 22:50:06] "GET /video HTTP/1.1" 200 -
127.0.0.1 - - [15/Aug/2020 22:50:09] "GET /proceso HTTP/1.1" 200 -
127.0.0.1 - - [15/Aug/2020 22:50:09] "GET /video HTTP/1.1" 200 -
127.0.0.1 - - [15/Aug/2020 22:50:49] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [15/Aug/2020 22:50:49] "GET /video HTTP/1.1" 200 -
127.0.0.1 - - [15/Aug/2020 22:50:52] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [15/Aug/2020 22:50:52] "GET /video HTTP/1.1" 200 -
127.0.0.1 - - [15/Aug/2020 22:50:54] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [15/Aug/2020 22:50:55] "GET /video HTTP/1.1" 200 -
